# Idea 1: Do the same architecture as the paper, but initialize the latent space of the TM with portuguese weights

Dependencies

In [ ]:
!pip install -q "transformers>=4.45.0" "datasets>=2.18.0" sacrebleu sentencepiece accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.9 MB/s eta 0:00:00


Import the models (These are simple encoder decoder models for the baseline)

In [ ]:
from transformers import AutoTokenizer

# Portuguese encoder LM
ENCODER_MODEL_NAME = "neuralmind/bert-base-portuguese-cased"

# English decoder + LM prior
DECODER_MODEL_NAME = "bert-base-uncased"

encoder_tokenizer = AutoTokenizer.from_pretrained(ENCODER_MODEL_NAME)
decoder_tokenizer = AutoTokenizer.from_pretrained(DECODER_MODEL_NAME)

print("Encoder vocab size:", encoder_tokenizer.vocab_size)
print("Decoder vocab size:", decoder_tokenizer.vocab_size)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Encoder vocab size: 29794
Decoder vocab size: 30522


In [ ]:
from transformers import EncoderDecoderModel

model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    ENCODER_MODEL_NAME,
    DECODER_MODEL_NAME,
)

# Decoder config for generation
model.config.decoder_start_token_id = decoder_tokenizer.cls_token_id
model.config.eos_token_id = decoder_tokenizer.sep_token_id
model.config.pad_token_id = decoder_tokenizer.pad_token_id

model.config.vocab_size = model.config.decoder.vocab_size

# Some default generation settings
model.config.max_length = 64
model.config.min_length = 1
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True

print("Model ready.")

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

Model ready.


Get the kriolu to english dataset (this is the KreyolMT one Kevin suggested)

In [ ]:
from datasets import load_dataset, DatasetDict

hf_ds = load_dataset("jhu-clsp/kreyol-mt", "kea-eng")

print(hf_ds)
print(hf_ds["train"][0])

README.md: 0.00B [00:00, ?B/s]

kea-eng/test-00000-of-00001.parquet:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

kea-eng/train-00000-of-00001.parquet:   0%|          | 0.00/193k [00:00<?, ?B/s]

kea-eng/validation-00000-of-00001.parque(…):   0%|          | 0.00/16.6k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/163 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1470 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/84 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 163
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1470
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 84
    })
})
{'translation': {'src_lang': 'kea', 'src_text': 'Dipôs di xinti konfortável ku formatason y edison di web, más tardi, bu pode kria bu própi website.', 'tgt_lang': 'eng', 'tgt_text': 'After you become comfortable with formatting and editing on the web, then later, you might create your own website.'}}


In [ ]:
from datasets import DatasetDict

def convert_batch(batch):
    trans = batch["translation"]
    kri = [ex["src_text"] for ex in trans]
    en  = [ex["tgt_text"] for ex in trans]
    return {"kri": kri, "en": en}

train_ds = hf_ds["train"].map(
    convert_batch,
    batched=True,
    remove_columns=["translation"],
)

val_ds = hf_ds["validation"].map(
    convert_batch,
    batched=True,
    remove_columns=["translation"],
)

test_ds = hf_ds["test"].map(
    convert_batch,
    batched=True,
    remove_columns=["translation"],
)

raw_datasets = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds,
})

raw_datasets, raw_datasets["train"][0]

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['kri', 'en'],
         num_rows: 1470
     })
     validation: Dataset({
         features: ['kri', 'en'],
         num_rows: 84
     })
     test: Dataset({
         features: ['kri', 'en'],
         num_rows: 163
     })
 }),
 {'kri': 'Dipôs di xinti konfortável ku formatason y edison di web, más tardi, bu pode kria bu própi website.',
  'en': 'After you become comfortable with formatting and editing on the web, then later, you might create your own website.'})

Tokenize and Collate the Data

In [ ]:
MAX_SOURCE_LEN = 64
MAX_TARGET_LEN = 64

def preprocess_batch(batch):
    # Kriolu -> Portuguese tokenizer
    src = encoder_tokenizer(
        batch["kri"],
        padding="max_length",
        truncation=True,
        max_length=MAX_SOURCE_LEN,
    )
    # English targets via English tokenizer
    with decoder_tokenizer.as_target_tokenizer():
        tgt = decoder_tokenizer(
            batch["en"],
            padding="max_length",
            truncation=True,
            max_length=MAX_TARGET_LEN,
        )

    labels = tgt["input_ids"]
    pad_id = decoder_tokenizer.pad_token_id
    labels = [
        [(tok if tok != pad_id else -100) for tok in seq]
        for seq in labels
    ]

    src["labels"] = labels
    return src

tokenized_datasets = raw_datasets.map(
    preprocess_batch,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets

Map:   0%|          | 0/1470 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1470
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 84
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 163
    })
})

In [ ]:
from dataclasses import dataclass
from typing import Dict, List
import torch

@dataclass
class DataCollatorForSeq2Seq:
    pad_token_id: int

    def __call__(self, features: List[Dict]):
        keys = features[0].keys()
        batch = {k: [f[k] for f in features] for k in keys}

        input_ids = torch.tensor(batch["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(batch["attention_mask"], dtype=torch.long)
        labels = torch.tensor(batch["labels"], dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

data_collator = DataCollatorForSeq2Seq(
    pad_token_id=encoder_tokenizer.pad_token_id
)

freeze the LM prior

In [ ]:
from transformers import BertForMaskedLM

DECODER_MODEL_NAME = "bert-base-uncased"

lm_prior = BertForMaskedLM.from_pretrained(DECODER_MODEL_NAME)

for p in lm_prior.parameters():
    p.requires_grad = False

lm_prior.eval()
print("LM prior loaded and frozen.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LM prior loaded and frozen.


train with the KL loss formula from the paper (I had Chat generate this one because its long af but all the key elements are there)

In [ ]:
import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
lm_prior.to(device)

BATCH_SIZE = 8

train_loader = DataLoader(
    tokenized_datasets["train"],
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator,
)

val_loader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator,
)

len(train_loader), len(val_loader)

(184, 11)

In [ ]:
import torch.nn.functional as F
from tqdm.auto import tqdm

lambda_kl = 0.5   # λ from the paper
tau = 2.0         # τ from the paper
EPOCHS = 30
LR = 5e-5

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

def train_one_epoch(epoch):
    model.train()
    total_loss = 0.0
    total_ce = 0.0
    total_kl = 0.0
    total_tokens = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)
    for batch in pbar:
        # Move batch to device
        for k in batch:
            batch[k] = batch[k].to(device)

        labels = batch["labels"]

        # Forward pass: MT model
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )
        ce_loss = outputs.loss
        logits_tm = outputs.logits  # [B, T, V]

        # Build LM inputs (targets), replace -100 with pad_id
        pad_id = decoder_tokenizer.pad_token_id
        lm_input_ids = torch.where(labels == -100, pad_id, labels)
        lm_attention_mask = (lm_input_ids != pad_id).long()

        with torch.no_grad():
            lm_outputs = lm_prior(
                input_ids=lm_input_ids,
                attention_mask=lm_attention_mask,
            )
            logits_lm = lm_outputs.logits  # [B, T, V]

        # KL term with temperature τ
        p_tm = F.softmax(logits_tm / tau, dim=-1)
        p_lm = F.softmax(logits_lm / tau, dim=-1)

        log_p_tm = F.log_softmax(logits_tm / tau, dim=-1)
        log_p_lm = F.log_softmax(logits_lm / tau, dim=-1)

        kl_per_token = torch.sum(
            p_tm * (log_p_tm - log_p_lm),
            dim=-1,
        )  # [B, T]

        # Mask real tokens (where labels != -100)
        active = (labels != -100).float()
        kl_sum = (kl_per_token * active).sum()
        active_count = active.sum().clamp(min=1.0)
        kl_mean = kl_sum / active_count

        kl_loss = (tau ** 2) * kl_mean  # τ² scaling like in the paper

        loss = ce_loss + lambda_kl * kl_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # stats
        total_loss += loss.item() * active_count.item()
        total_ce += ce_loss.item() * active_count.item()
        total_kl += kl_loss.item() * active_count.item()
        total_tokens += active_count.item()

        avg_loss = total_loss / total_tokens
        avg_ce = total_ce / total_tokens
        avg_kl = total_kl / total_tokens
        pbar.set_postfix(
            loss=f"{avg_loss:.4f}",
            ce=f"{avg_ce:.4f}",
            kl=f"{avg_kl:.4f}",
        )

    return total_loss / total_tokens

@torch.no_grad()
def evaluate_loss():
    model.eval()
    total_loss = 0.0
    total_tokens = 0.0

    for batch in val_loader:
        for k in batch:
            batch[k] = batch[k].to(device)
        labels = batch["labels"]
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )
        loss = outputs.loss
        active = (labels != -100).float()
        total_loss += loss.item() * active.sum().item()
        total_tokens += active.sum().item()

    return total_loss / max(total_tokens, 1.0)


In [ ]:
# Run training
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(epoch)
    val_loss = evaluate_loss()
    print(f"Epoch {epoch+1} | train loss: {train_loss:.4f} | val loss: {val_loss:.4f}")

Epoch 1:   0%|          | 0/184 [00:00<?, ?it/s]

Epoch 1 | train loss: 1.7111 | val loss: 5.8158


Epoch 2:   0%|          | 0/184 [00:00<?, ?it/s]

Epoch 2 | train loss: 1.6249 | val loss: 5.9176


Epoch 3:   0%|          | 0/184 [00:00<?, ?it/s]

...

Epoch 28 | train loss: 0.8346 | val loss: 5.8687


Epoch 29:   0%|          | 0/184 [00:00<?, ?it/s]

Epoch 29 | train loss: 0.8047 | val loss: 5.8757


Epoch 30:   0%|          | 0/184 [00:00<?, ?it/s]

Epoch 30 | train loss: 0.7834 | val loss: 5.9138


testing using the same way the original guys did it

In [ ]:
import sacrebleu
import torch

def translate_texts(texts_kri, max_len=64):
    model.eval()
    device = next(model.parameters()).device

    # Make sure it's a list of plain Python strings
    if isinstance(texts_kri, str):
        texts_kri = [texts_kri]
    else:
        texts_kri = [str(x) for x in texts_kri]

    enc = encoder_tokenizer(
        texts_kri,
        padding=True,
        truncation=True,
        max_length=MAX_SOURCE_LEN,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        gen_ids = model.generate(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"],
            max_length=max_len,
            num_beams=4,
        )

    return decoder_tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

def evaluate_split(split="validation", max_samples=None):
    ds = raw_datasets[split]
    if max_samples is not None:
        ds = ds.select(range(min(max_samples, len(ds))))

    # Explicitly convert to list[str]
    kri = [str(x) for x in ds["kri"]]
    refs = [str(x) for x in ds["en"]]

    hyps = translate_texts(kri)

    bleu = sacrebleu.corpus_bleu(hyps, [refs])
    print(f"{split} BLEU: {bleu.score:.2f}")
    return hyps, refs

val_hyps, val_refs = evaluate_split("validation", max_samples=84)

if "test" in raw_datasets:
    test_hyps, test_refs = evaluate_split("test", max_samples=163)

validation BLEU: 4.64
test BLEU: 3.02


# Idea 1.1: Add a Portuguese TM as another prior

import the portugues TM (I found this one on hugging face)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

teacher_model_name = "manueldeprada/t5-small-pt-en"  # PT→EN T5-small

pt_en_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
pt_en_model = AutoModelForSeq2SeqLM.from_pretrained(teacher_model_name).to(device)

for p in pt_en_model.parameters():
    p.requires_grad = False

pt_en_model.eval()
print("Portuguese→English teacher TM loaded.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Portuguese→English teacher TM loaded.


make the dataloaders

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE_TM_PRIOR = 8
def collate_raw(batch):
    kri = [b["kri"] for b in batch]
    en  = [b["en"] for b in batch]
    return {"kri": kri, "en": en}

raw_train_loader = DataLoader(
    raw_datasets["train"],
    batch_size=BATCH_SIZE_TM_PRIOR,
    shuffle=True,
    collate_fn=collate_raw,
)

raw_val_loader = DataLoader(
    raw_datasets["validation"],
    batch_size=BATCH_SIZE_TM_PRIOR,
    shuffle=False,
    collate_fn=collate_raw,
)

len(raw_train_loader), len(raw_val_loader)

(184, 11)

Im lazy, so I had chat alter the training loop we have above to include this TM as a prior in the KL loss

In [ ]:
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

lambda_lm = 0.5   # weight for LM prior KL
tau = 2.0         # temperature for LM prior
lambda_tm = 0.5   # weight for TM prior (teacher) CE

EPOCHS_TM_PRIOR = 15   # e.g., 3–5 epochs for this second stage
LR_TM_PRIOR = 5e-5    # smaller LR for fine-tuning

optimizer_tm_prior = torch.optim.AdamW(model.parameters(), lr=LR_TM_PRIOR)

def make_label_tensor(texts_en, max_len=MAX_TARGET_LEN):
    # Tokenize EN text with your decoder tokenizer and map PAD → -100
    with decoder_tokenizer.as_target_tokenizer():
        tgt = decoder_tokenizer(
            texts_en,
            padding="max_length",
            truncation=True,
            max_length=max_len,
        )
    pad_id = decoder_tokenizer.pad_token_id
    labels = [
        [(tok if tok != pad_id else -100) for tok in seq]
        for seq in tgt["input_ids"]
    ]
    return torch.tensor(labels, dtype=torch.long, device=device)

def train_one_epoch_tm_prior(epoch):
    model.train()
    total_loss = total_gold = total_kl = total_teacher = 0.0
    total_tokens = 0.0

    pbar = tqdm(raw_train_loader, desc=f"TM prior Epoch {epoch+1}", leave=False)
    for batch in pbar:
        kri_texts = batch["kri"]
        en_gold   = batch["en"]

        # 1) Tokenize source Kriolu with PT encoder tokenizer (your encoder)
        src_enc = encoder_tokenizer(
            kri_texts,
            padding="max_length",
            truncation=True,
            max_length=MAX_SOURCE_LEN,
            return_tensors="pt",
        ).to(device)

        # 2) Gold EN labels
        gold_labels = make_label_tensor(en_gold, max_len=MAX_TARGET_LEN)

        # 3) Teacher translations (PT→EN TM, treating Kriolu as PT-like)
        with torch.no_grad():
            teacher_inputs = pt_en_tokenizer(
                kri_texts,
                padding=True,
                truncation=True,
                return_tensors="pt",
            ).to(device)

            teacher_gen_ids = pt_en_model.generate(
                **teacher_inputs,
                max_length=MAX_TARGET_LEN,
                num_beams=4,
            )

            teacher_texts = pt_en_tokenizer.batch_decode(
                teacher_gen_ids,
                skip_special_tokens=True,
            )

        teacher_labels = make_label_tensor(teacher_texts, max_len=MAX_TARGET_LEN)

        # 4) Forward pass through your Kriolu→EN TM
        #    IMPORTANT: pass labels=gold_labels so the decoder gets valid inputs
        outputs = model(
            input_ids=src_enc["input_ids"],
            attention_mask=src_enc["attention_mask"],
            labels=gold_labels,
        )
        logits_tm = outputs.logits  # [B, T, V]
        V = logits_tm.size(-1)

        # --- 4a) CE vs gold labels (use model's own loss) ---
        gold_ce = outputs.loss

        # --- 4b) LM prior KL (same structure as earlier loop) ---
        pad_id = decoder_tokenizer.pad_token_id
        lm_input_ids = torch.where(gold_labels == -100, pad_id, gold_labels)
        lm_attention_mask = (lm_input_ids != pad_id).long()

        with torch.no_grad():
            lm_outputs = lm_prior(
                input_ids=lm_input_ids,
                attention_mask=lm_attention_mask,
            )
            logits_lm = lm_outputs.logits  # [B, T, V]

        p_tm = F.softmax(logits_tm / tau, dim=-1)
        p_lm = F.softmax(logits_lm / tau, dim=-1)
        log_p_tm = F.log_softmax(logits_tm / tau, dim=-1)
        log_p_lm = F.log_softmax(logits_lm / tau, dim=-1)

        kl_per_token = torch.sum(
            p_tm * (log_p_tm - log_p_lm),
            dim=-1,
        )  # [B, T]

        active = (gold_labels != -100).float()
        kl_sum = (kl_per_token * active).sum()
        active_count = active.sum().clamp(min=1.0)
        kl_mean = kl_sum / active_count
        kl_loss = (tau ** 2) * kl_mean

        # --- 4c) CE vs teacher labels (TM prior) ---
        teacher_ce = F.cross_entropy(
            logits_tm.view(-1, V),
            teacher_labels.view(-1),
            ignore_index=-100,
        )

        # Combine losses
        loss = gold_ce + lambda_lm * kl_loss + lambda_tm * teacher_ce

        optimizer_tm_prior.zero_grad()
        loss.backward()
        optimizer_tm_prior.step()

        # bookkeeping
        total_loss     += loss.item()      * active_count.item()
        total_gold     += gold_ce.item()   * active_count.item()
        total_kl       += kl_loss.item()   * active_count.item()
        total_teacher  += teacher_ce.item()* active_count.item()
        total_tokens   += active_count.item()

        avg_loss    = total_loss / total_tokens
        avg_gold    = total_gold / total_tokens
        avg_kl      = total_kl   / total_tokens
        avg_teacher = total_teacher / total_tokens

        pbar.set_postfix(
            loss=f"{avg_loss:.4f}",
            gold=f"{avg_gold:.4f}",
            kl=f"{avg_kl:.4f}",
            tm=f"{avg_teacher:.4f}",
        )

    return total_loss / total_tokens

@torch.no_grad()
def evaluate_loss_tm_prior():
    # quick val CE (gold only) to track overfitting
    model.eval()
    total_loss = 0.0
    total_tokens = 0.0

    for batch in raw_val_loader:
        kri_texts = batch["kri"]
        en_gold   = batch["en"]

        src_enc = encoder_tokenizer(
            kri_texts,
            padding="max_length",
            truncation=True,
            max_length=MAX_SOURCE_LEN,
            return_tensors="pt",
        ).to(device)

        gold_labels = make_label_tensor(en_gold, max_len=MAX_TARGET_LEN)

        outputs = model(
            input_ids=src_enc["input_ids"],
            attention_mask=src_enc["attention_mask"],
            labels=gold_labels,
        )
        loss = outputs.loss

        active = (gold_labels != -100).float()
        total_loss += loss.item() * active.sum().item()
        total_tokens += active.sum().item()

    return total_loss / max(total_tokens, 1.0)


execute the training loop above

In [ ]:
for epoch in range(EPOCHS_TM_PRIOR):
    train_loss = train_one_epoch_tm_prior(epoch)
    val_loss = evaluate_loss_tm_prior()
    print(f"[TM prior] Epoch {epoch+1} | train loss: {train_loss:.4f} | val loss: {val_loss:.4f}")

TM prior Epoch 1:   0%|          | 0/184 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:555: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:575: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decode

[TM prior] Epoch 1 | train loss: 4.3940 | val loss: 5.5387


TM prior Epoch 2:   0%|          | 0/184 [00:00<?, ?it/s]

[TM prior] Epoch 2 | train loss: 4.3635 | val loss: 5.4529


...

[TM prior] Epoch 13 | train loss: 3.4319 | val loss: 5.4878


TM prior Epoch 14:   0%|          | 0/184 [00:00<?, ?it/s]

[TM prior] Epoch 14 | train loss: 3.3737 | val loss: 5.5609


TM prior Epoch 15:   0%|          | 0/184 [00:00<?, ?it/s]

[TM prior] Epoch 15 | train loss: 3.3431 | val loss: 5.5603


just re-run the same BLEU stuff above

In [ ]:
val_hyps, val_refs = evaluate_split("validation", max_samples=84)

if "test" in raw_datasets:
    test_hyps, test_refs = evaluate_split("test", max_samples=163)

validation BLEU: 5.22
test BLEU: 2.44


#BASELINE: Latent space is randomly initialized

In [ ]:
# Small-ish random BERT-like encoder/decoder so it's trainable from scratch
enc_config_scratch = BertConfig(
    vocab_size=encoder_tokenizer.vocab_size,
    hidden_size=256,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=1024,
    max_position_embeddings=128,
)

dec_config_scratch = BertConfig(
    vocab_size=decoder_tokenizer.vocab_size,
    hidden_size=256,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=1024,
    max_position_embeddings=128,
    is_decoder=True,
    add_cross_attention=True,  # decoder attends to encoder
)

scratch_encoder = BertModel(enc_config_scratch)
scratch_decoder = BertLMHeadModel(dec_config_scratch)

scratch_lm_model = EncoderDecoderModel(
    encoder=scratch_encoder,
    decoder=scratch_decoder,
)

# Generation / special token setup
scratch_lm_model.config.decoder_start_token_id = decoder_tokenizer.cls_token_id
scratch_lm_model.config.eos_token_id = decoder_tokenizer.sep_token_id
scratch_lm_model.config.pad_token_id = decoder_tokenizer.pad_token_id
scratch_lm_model.config.vocab_size = decoder_tokenizer.vocab_size

scratch_lm_model.config.max_length = 64
scratch_lm_model.config.min_length = 1
scratch_lm_model.config.no_repeat_ngram_size = 3
scratch_lm_model.config.early_stopping = True

scratch_lm_model.to(device)
print("Randomly initialized NMT model (with LM prior) ready.")

Randomly initialized NMT model (with LM prior) ready.


In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE_SCRATCH = 8

scratch_train_loader = DataLoader(
    tokenized_datasets["train"],
    batch_size=BATCH_SIZE_SCRATCH,
    shuffle=True,
    collate_fn=data_collator,
)

scratch_val_loader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=BATCH_SIZE_SCRATCH,
    shuffle=False,
    collate_fn=data_collator,
)

len(scratch_train_loader), len(scratch_val_loader)

(184, 11)

In [ ]:
import torch.nn.functional as F
from tqdm.auto import tqdm

lambda_lm = 0.5   # weight for LM prior KL (like the paper)
tau = 2.0         # temperature for LM prior

EPOCHS_SCRATCH = 30   # you can start with 10, then go to 30
LR_SCRATCH = 5e-4     # a bit higher than fine-tune LR

optimizer_scratch = torch.optim.AdamW(scratch_lm_model.parameters(), lr=LR_SCRATCH)

def train_one_epoch_scratch(epoch):
    scratch_lm_model.train()
    total_loss = 0.0
    total_ce = 0.0
    total_kl = 0.0
    total_tokens = 0.0

    pbar = tqdm(scratch_train_loader, desc=f"[Scratch+LM] Epoch {epoch+1}", leave=False)
    for batch in pbar:
        for k in batch:
            batch[k] = batch[k].to(device)

        labels = batch["labels"]

        # Forward pass: CE loss from the encoder-decoder
        outputs = scratch_lm_model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )
        ce_loss = outputs.loss
        logits_tm = outputs.logits  # [B, T, V]

        # LM prior: use gold labels as input to the LM
        pad_id = decoder_tokenizer.pad_token_id
        lm_input_ids = torch.where(labels == -100, pad_id, labels)
        lm_attention_mask = (lm_input_ids != pad_id).long()

        with torch.no_grad():
            lm_outputs = lm_prior(
                input_ids=lm_input_ids.to(device),
                attention_mask=lm_attention_mask.to(device),
            )
            logits_lm = lm_outputs.logits  # [B, T, V]

        # KL(p_tm || p_lm) with temperature τ
        p_tm = F.softmax(logits_tm / tau, dim=-1)
        p_lm = F.softmax(logits_lm / tau, dim=-1)
        log_p_tm = F.log_softmax(logits_tm / tau, dim=-1)
        log_p_lm = F.log_softmax(logits_lm / tau, dim=-1)

        kl_per_token = torch.sum(
            p_tm * (log_p_tm - log_p_lm),
            dim=-1,
        )  # [B, T]

        active = (labels != -100).float()
        kl_sum = (kl_per_token * active).sum()
        active_count = active.sum().clamp(min=1.0)
        kl_mean = kl_sum / active_count
        kl_loss = (tau ** 2) * kl_mean

        loss = ce_loss + lambda_lm * kl_loss

        optimizer_scratch.zero_grad()
        loss.backward()
        optimizer_scratch.step()

        # bookkeeping
        total_loss += loss.item() * active_count.item()
        total_ce   += ce_loss.item() * active_count.item()
        total_kl   += kl_loss.item() * active_count.item()
        total_tokens += active_count.item()

        avg_loss = total_loss / total_tokens
        avg_ce   = total_ce   / total_tokens
        avg_kl   = total_kl   / total_tokens

        pbar.set_postfix(
            loss=f"{avg_loss:.4f}",
            ce=f"{avg_ce:.4f}",
            kl=f"{avg_kl:.4f}",
        )

    return total_loss / total_tokens

@torch.no_grad()
def evaluate_loss_scratch():
    scratch_lm_model.eval()
    total_loss = 0.0
    total_tokens = 0.0

    for batch in scratch_val_loader:
        for k in batch:
            batch[k] = batch[k].to(device)
        labels = batch["labels"]

        outputs = scratch_lm_model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )
        loss = outputs.loss

        active = (labels != -100).float()
        total_loss += loss.item() * active.sum().item()
        total_tokens += active.sum().item()

    return total_loss / max(total_tokens, 1.0)

# Run training for the scratch+LM-prior TM
for epoch in range(EPOCHS_SCRATCH):
    train_loss = train_one_epoch_scratch(epoch)
    val_loss = evaluate_loss_scratch()
    print(f"[Scratch+LM] Epoch {epoch+1} | train loss: {train_loss:.4f} | val loss: {val_loss:.4f}")


[Scratch+LM] Epoch 1:   0%|          | 0/184 [00:00<?, ?it/s]

[Scratch+LM] Epoch 1 | train loss: 10.2751 | val loss: 5.7306


[Scratch+LM] Epoch 2:   0%|          | 0/184 [00:00<?, ?it/s]

[Scratch+LM] Epoch 2 | train loss: 9.6058 | val loss: 5.7106


[Scratch+LM] Epoch 3:   0%|          | 0/184 [00:00<?, ?it/s]

...

[Scratch+LM] Epoch 28 | train loss: 1.9073 | val loss: 8.6810


[Scratch+LM] Epoch 29:   0%|          | 0/184 [00:00<?, ?it/s]

[Scratch+LM] Epoch 29 | train loss: 1.8526 | val loss: 8.6769


[Scratch+LM] Epoch 30:   0%|          | 0/184 [00:00<?, ?it/s]

[Scratch+LM] Epoch 30 | train loss: 1.7885 | val loss: 8.8795


In [ ]:
import sacrebleu

def translate_texts_scratch_lm(texts_kri, max_len=64):
    scratch_lm_model.eval()
    device = next(scratch_lm_model.parameters()).device

    if isinstance(texts_kri, str):
        texts_kri = [texts_kri]
    else:
        texts_kri = [str(x) for x in texts_kri]

    enc = encoder_tokenizer(
        texts_kri,
        padding=True,
        truncation=True,
        max_length=MAX_SOURCE_LEN,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        gen_ids = scratch_lm_model.generate(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"],
            max_length=max_len,
            num_beams=4,
        )

    return decoder_tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

def evaluate_split_scratch_lm(split="validation", max_samples=None):
    ds = raw_datasets[split]
    if max_samples is not None:
        ds = ds.select(range(min(max_samples, len(ds))))

    kri = [str(x) for x in ds["kri"]]
    refs = [str(x) for x in ds["en"]]

    hyps = translate_texts_scratch_lm(kri)

    bleu = sacrebleu.corpus_bleu(hyps, [refs])
    print(f"[Scratch+LM] {split} BLEU: {bleu.score:.2f}")
    return hyps, refs

scratch_lm_val_hyps, scratch_lm_val_refs = evaluate_split_scratch_lm("validation", max_samples=84)

if "test" in raw_datasets:
    scratch_lm_test_hyps, scratch_lm_test_refs = evaluate_split_scratch_lm("test", max_samples=163)


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[Scratch+LM] validation BLEU: 5.44
[Scratch+LM] test BLEU: 2.15


# NEW BASELINE

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import sacrebleu

# -----------------------------------------------------
# Positional encoding (standard sinusoidal PE)
# -----------------------------------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 128):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)  # [max_len, 1, d_model]
        self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [T, B, d_model]
        """
        return x + self.pe[: x.size(0)]


# -----------------------------------------------------
# Simple Transformer encoder–decoder MT model from scratch
# -----------------------------------------------------
class SimpleTransformerMT(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        d_model=256,
        nhead=4,
        num_encoder_layers=3,
        num_decoder_layers=3,
        dim_feedforward=1024,
        dropout=0.1,
        pad_id_src=0,
        pad_id_tgt=0,
        bos_id=None,
        eos_id=None,
        max_len=128,
    ):
        super().__init__()
        self.d_model = d_model

        self.src_embed = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embed = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len=max_len)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False,  # we use [T, B, E]
        )

        self.output_proj = nn.Linear(d_model, tgt_vocab_size)

        self.pad_id_src = pad_id_src
        self.pad_id_tgt = pad_id_tgt
        # Use CLS as BOS and SEP as EOS like your other model
        self.bos_id = bos_id if bos_id is not None else pad_id_tgt
        self.eos_id = eos_id if eos_id is not None else pad_id_tgt

    def _encode(self, input_ids, attention_mask=None):
        """
        input_ids: [B, S]
        attention_mask: [B, S] or None
        """
        src = self.src_embed(input_ids) * math.sqrt(self.d_model)  # [B, S, E]
        src = src.transpose(0, 1)  # [S, B, E]
        src = self.pos_enc(src)

        if attention_mask is not None:
            src_key_padding_mask = attention_mask == 0  # True where padding
        else:
            src_key_padding_mask = input_ids == self.pad_id_src

        memory = self.transformer.encoder(
            src,
            src_key_padding_mask=src_key_padding_mask,
        )
        return memory, src_key_padding_mask

    def _prepare_decoder_inputs(self, labels):
        """
        labels: [B, T] with -100 for ignore_index.
        Returns:
          decoder_input_ids: [B, T]
          labels_in: [B, T] with pad_id instead of -100
        """
        labels_in = labels.clone()
        labels_in[labels_in == -100] = self.pad_id_tgt
        B, T = labels_in.shape
        decoder_input_ids = torch.full_like(labels_in, fill_value=self.pad_id_tgt)
        decoder_input_ids[:, 0] = self.bos_id
        if T > 1:
            decoder_input_ids[:, 1:] = labels_in[:, :-1]
        return decoder_input_ids, labels_in

    def forward(self, input_ids, attention_mask=None, labels=None):
        """
        Training forward:
          input_ids: [B, S]
          attention_mask: [B, S] or None
          labels: [B, T] with -100 as ignore_index
        Returns:
          object with .loss (CE) and .logits [B, T, V]
        """
        memory, src_key_padding_mask = self._encode(input_ids, attention_mask)

        if labels is None:
            raise ValueError("SimpleTransformerMT.forward is training-only (labels required).")

        decoder_input_ids, labels_in = self._prepare_decoder_inputs(labels)

        tgt = self.tgt_embed(decoder_input_ids) * math.sqrt(self.d_model)  # [B, T, E]
        tgt = tgt.transpose(0, 1)  # [T, B, E]
        tgt = self.pos_enc(tgt)

        tgt_key_padding_mask = decoder_input_ids == self.pad_id_tgt  # [B, T]
        T = decoder_input_ids.size(1)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(T).to(input_ids.device)

        out = self.transformer.decoder(
            tgt,
            memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=src_key_padding_mask,
        )  # [T, B, E]

        logits = self.output_proj(out)  # [T, B, V]
        logits = logits.transpose(0, 1)  # [B, T, V]

        V = logits.size(-1)
        ce_loss = F.cross_entropy(
            logits.reshape(-1, V),
            labels.reshape(-1),
            ignore_index=-100,
        )

        # match HF-style output: .loss and .logits
        return type("MTOutput", (), {"loss": ce_loss, "logits": logits})

    @torch.no_grad()
    def generate(self, input_ids, attention_mask=None, max_length=64):
        """
        Greedy decode:
          input_ids: [B, S]
        Returns:
          generated_ids: [B, <= max_length]
        """
        device = input_ids.device
        memory, src_key_padding_mask = self._encode(input_ids, attention_mask)
        B = input_ids.size(0)

        decoder_input_ids = torch.full(
            (B, 1), self.bos_id, dtype=torch.long, device=device
        )
        finished = torch.zeros(B, dtype=torch.bool, device=device)
        generated_tokens = []

        for step in range(max_length):
            tgt = self.tgt_embed(decoder_input_ids) * math.sqrt(self.d_model)  # [B, t, E]
            tgt = tgt.transpose(0, 1)  # [t, B, E]
            tgt = self.pos_enc(tgt)

            t_len = decoder_input_ids.size(1)
            tgt_mask = nn.Transformer.generate_square_subsequent_mask(t_len).to(device)
            tgt_key_padding_mask = decoder_input_ids == self.pad_id_tgt

            out = self.transformer.decoder(
                tgt,
                memory,
                tgt_mask=tgt_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=src_key_padding_mask,
            )  # [t, B, E]

            logits = self.output_proj(out)  # [t, B, V]
            next_logits = logits[-1]  # [B, V]
            next_tokens = next_logits.argmax(dim=-1)  # [B]

            generated_tokens.append(next_tokens)
            decoder_input_ids = torch.cat(
                [decoder_input_ids, next_tokens.unsqueeze(1)], dim=1
            )

            finished = finished | (next_tokens == self.eos_id)
            if finished.all():
                break

        if not generated_tokens:
            return torch.full((B, 1), self.eos_id, dtype=torch.long, device=device)

        gen = torch.stack(generated_tokens, dim=1)  # [B, T_dec]
        return gen


# -----------------------------------------------------
# Instantiate baseline model (all weights random)
# -----------------------------------------------------
src_vocab_size = encoder_tokenizer.vocab_size
tgt_vocab_size = decoder_tokenizer.vocab_size

pad_id_src = encoder_tokenizer.pad_token_id
pad_id_tgt = decoder_tokenizer.pad_token_id
bos_id = decoder_tokenizer.cls_token_id or pad_id_tgt
eos_id = (decoder_tokenizer.sep_token_id
          or decoder_tokenizer.eos_token_id
          or pad_id_tgt)

baseline_model = SimpleTransformerMT(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=256,
    nhead=4,
    num_encoder_layers=3,
    num_decoder_layers=3,
    dim_feedforward=1024,
    dropout=0.1,
    pad_id_src=pad_id_src,
    pad_id_tgt=pad_id_tgt,
    bos_id=bos_id,
    eos_id=eos_id,
    max_len=max(MAX_SOURCE_LEN, MAX_TARGET_LEN, 128),
)

baseline_model.to(device)
lm_prior.to(device)  # reused from above
print("Baseline raw Transformer MT model (with LM prior) ready.")


Baseline raw Transformer MT model (with LM prior) ready.


In [ ]:
# -----------------------------------------------------
# DataLoaders for baseline
# -----------------------------------------------------
BATCH_SIZE_BASELINE = 8

baseline_train_loader = DataLoader(
    tokenized_datasets["train"],
    batch_size=BATCH_SIZE_BASELINE,
    shuffle=True,
    collate_fn=data_collator,
)

baseline_val_loader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=BATCH_SIZE_BASELINE,
    shuffle=False,
    collate_fn=data_collator,
)

len(baseline_train_loader), len(baseline_val_loader)


# -----------------------------------------------------
# Training: CE + LM prior KL (same λ and τ as main model)
# -----------------------------------------------------
# Reuse lambda_kl and tau defined above for LM prior,
# so the baseline is directly comparable.
EPOCHS_BASELINE = 30
LR_BASELINE = 5e-5

optimizer_baseline = torch.optim.AdamW(baseline_model.parameters(), lr=LR_BASELINE)

def train_one_epoch_baseline(epoch):
    baseline_model.train()
    total_loss = 0.0
    total_ce = 0.0
    total_kl = 0.0
    total_tokens = 0.0

    pbar = tqdm(baseline_train_loader, desc=f"[Baseline raw+LM] Epoch {epoch+1}", leave=False)
    for batch in pbar:
        for k in batch:
            batch[k] = batch[k].to(device)

        labels = batch["labels"]

        # 1) Translation CE loss from raw Transformer
        outputs = baseline_model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )
        ce_loss = outputs.loss
        logits_tm = outputs.logits  # [B, T, V]

        # 2) LM prior KL using same lm_prior as before
        pad_id = pad_id_tgt
        lm_input_ids = torch.where(labels == -100, pad_id, labels)
        lm_attention_mask = (lm_input_ids != pad_id).long()

        with torch.no_grad():
            lm_outputs = lm_prior(
                input_ids=lm_input_ids,
                attention_mask=lm_attention_mask,
            )
            logits_lm = lm_outputs.logits  # [B, T, V]

        p_tm = F.softmax(logits_tm / tau, dim=-1)
        p_lm = F.softmax(logits_lm / tau, dim=-1)
        log_p_tm = F.log_softmax(logits_tm / tau, dim=-1)
        log_p_lm = F.log_softmax(logits_lm / tau, dim=-1)

        kl_per_token = torch.sum(
            p_tm * (log_p_tm - log_p_lm),
            dim=-1,
        )  # [B, T]

        active = (labels != -100).float()
        kl_sum = (kl_per_token * active).sum()
        active_count = active.sum().clamp(min=1.0)
        kl_mean = kl_sum / active_count
        kl_loss = (tau ** 2) * kl_mean

        loss = ce_loss + lambda_kl * kl_loss

        optimizer_baseline.zero_grad()
        loss.backward()
        optimizer_baseline.step()

        total_loss += loss.item() * active_count.item()
        total_ce += ce_loss.item() * active_count.item()
        total_kl += kl_loss.item() * active_count.item()
        total_tokens += active_count.item()

        avg_loss = total_loss / total_tokens
        avg_ce = total_ce / total_tokens
        avg_kl = total_kl / total_tokens

        pbar.set_postfix(
            loss=f"{avg_loss:.4f}",
            ce=f"{avg_ce:.4f}",
            kl=f"{avg_kl:.4f}",
        )

    return total_loss / total_tokens


@torch.no_grad()
def evaluate_loss_baseline():
    baseline_model.eval()
    total_loss = 0.0
    total_tokens = 0.0

    for batch in baseline_val_loader:
        for k in batch:
            batch[k] = batch[k].to(device)
        labels = batch["labels"]

        outputs = baseline_model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels,
        )
        loss = outputs.loss

        active = (labels != -100).float()
        total_loss += loss.item() * active.sum().item()
        total_tokens += active.sum().item()

    return total_loss / max(total_tokens, 1.0)


# Run baseline training
for epoch in range(EPOCHS_BASELINE):
    train_loss = train_one_epoch_baseline(epoch)
    val_loss = evaluate_loss_baseline()
    print(f"[Baseline raw+LM] Epoch {epoch+1} | train loss: {train_loss:.4f} | val loss: {val_loss:.4f}")


[Baseline raw+LM] Epoch 1:   0%|          | 0/184 [00:00<?, ?it/s]

[Baseline raw+LM] Epoch 1 | train loss: 3.9916 | val loss: 7.6838


[Baseline raw+LM] Epoch 2:   0%|          | 0/184 [00:00<?, ?it/s]

[Baseline raw+LM] Epoch 2 | train loss: 3.9152 | val loss: 7.7545


[Baseline raw+LM] Epoch 3:   0%|          | 0/184 [00:00<?, ?it/s]

...

[Baseline raw+LM] Epoch 28 | train loss: 2.8555 | val loss: 8.6169


[Baseline raw+LM] Epoch 29:   0%|          | 0/184 [00:00<?, ?it/s]

[Baseline raw+LM] Epoch 29 | train loss: 2.8340 | val loss: 8.6203


[Baseline raw+LM] Epoch 30:   0%|          | 0/184 [00:00<?, ?it/s]

[Baseline raw+LM] Epoch 30 | train loss: 2.8027 | val loss: 8.6959


In [ ]:
import sacrebleu
import torch

def translate_texts(texts_kri, max_len=64):
    model.eval()
    device = next(model.parameters()).device

    # Make sure it's a list of plain Python strings
    if isinstance(texts_kri, str):
        texts_kri = [texts_kri]
    else:
        texts_kri = [str(x) for x in texts_kri]

    enc = encoder_tokenizer(
        texts_kri,
        padding=True,
        truncation=True,
        max_length=MAX_SOURCE_LEN,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        gen_ids = model.generate(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"],
            max_length=max_len,
            num_beams=4,
        )

    return decoder_tokenizer.batch_decode(gen_ids, skip_special_tokens=True)

def evaluate_split(split="validation", max_samples=None):
    ds = raw_datasets[split]
    if max_samples is not None:
        ds = ds.select(range(min(max_samples, len(ds))))

    # Explicitly convert to list[str]
    kri = [str(x) for x in ds["kri"]]
    refs = [str(x) for x in ds["en"]]

    hyps = translate_texts(kri)

    bleu = sacrebleu.corpus_bleu(hyps, [refs])
    print(f"{split} BLEU: {bleu.score:.2f}")
    return hyps, refs

baseline_val_hyps, baseline_val_refs = evaluate_split("validation", max_samples=84)

if "test" in raw_datasets:
    baseline_test_hyps, baseline_test_refs = evaluate_split("test", max_samples=163)

validation BLEU: 0.12
test BLEU: 0.08
